# **MODELIZACIÓN DEL BRILLO DEL CIELO**
Conocido el valor de $\mu_{sky}$ de cada filtro se calcula la variación con la emisión de la atmósfera (dependiente de la distancia al cénit) y el brillo de la Luna (dependiente de su posición y su fase).

En términos de magnitudes, la variación del brillo del cielo con el ángulo cenital viene dado por (sin tener en cuenta el brillo de la Luna):

$\mu_{sky}(z)=\mu_{sky}-2.5\log_{10}\chi+\kappa(\chi-1)$.

Donde $\chi$ es la masa de aire (que contribuye al scattering, y por lo tanto al brillo del cielo) y $\kappa$ es el coeficiente de extinción del cielo en magnitudes por masa de aire.

La masa de aire se calcula como:

$\chi = (1-0.972\sin^2(z))^{-0.5}$,

donde $z$ es el ángulo cenital del objeto (*TFG de Jose María Salcedo*). 

Para poder añadir la variación producida por el brillo de la Luna es necesario pasar esta cantidad a nano Lamberts usando la transformación dada por Krisciunas ($B_{sky}$), ya que la ecuación que permite obtener $B_{moon}$ (brillo de la Luna en nano Lamberts) en función del ángulo de fase lunar y la posición de la luna (ángulo cenital y distancia al objeto) tiene las constantes calibradas para esta unidad. Una vez calculado $B_{moon}$, se calcula el cambio en la magnitud del brillo del cielo como:

$\Delta\mu=-2.5\log_{10}\left[\frac{B_{moon}+B_{sky}}{B_{sky}}\right]$






In [0]:
import numpy as np
import math
import astropy.units as u #lo utilizamos para decirle al programa con qué unidades estamos trabajando
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.coordinates import get_moon
from astropy.time import Time
from astropy.utils import iers
from matplotlib import pyplot as plt
iers.conf.auto_max_age = None
iers.conf.auto_download = False


class Target:  

    def __init__ (self, name, RA, DEC):
        # Entrada: nombre de la estrella, RA: ascensión recta (hh:mm:ss), DEC: declinación (dd:mm:ss),prioridad (A,B o C), tiempo de exposición, minima distania angular con la luna (grados)
        self.coord = SkyCoord(ra = RA, dec = DEC)
        self.name = name
        #self.priority = priority
        #self.exposure_time_h = exposure_time_h
        #self.minimum_lunar_distance_deg = minimum_lunar_distance_deg
    
    def local_coordinates (self, time, earth_location):
        # Entrada: fecha de observación, localización del observatorio
        # Salida: coordenadas horizontales (altura y azimut) en grados, minutos y segundos
        a = self.coord.transform_to(AltAz(obstime = time, location = earth_location))
        return a.alt, a.az   

    def lunar_distance_deg (self, time, moon):
        # Salida: separación angular (deg) entre la luna y el objeto
        moon_data = moon.separation(self.coord)
        return moon_data.deg

    def lunar_distance_rad (self, time, moon):
        # Salida: separación angular (rad) entre la luna y el objeto
        moon_data = moon.separation(self.coord)
        return moon_data.rad

    def air_mass_too_high (self, time, earth_location):
        # Salida: True si la altura del objeto es menor que 30º
        (a,b) = self.local_coordinates(time, earth_location)
        return a.deg < 30

    # Ángulo cenital en grados
    def zenith_angle_rad(self, time, earth_location):
        # Entrada: fecha de observación, localización del observatorio, objeto
        # Salida: ángulo cenital en grados, minutos y segundos
        a = self.coord.transform_to(AltAz(obstime = time, location = earth_location))
        z = np.pi/2-a.alt.rad
        return z 

    # Ángulo cenital en radianes
    def zenith_angle_deg(self, time, utc_offset, earth_location):
        # Entrada: fecha de observación, localización del observatorio, objeto
        # Salida: ángulo cenital en radianes
        a = self.coord.transform_to(AltAz(obstime = time, location = earth_location))
        z = 90-a.alt.deg
        return z       


    def brillo_cielo(self, location_observatory, time, ms, extinction_coef):
      # Cálculo del ángulo de la fase lunar
      # 0 para luna llena
      # 180 para luna nueva
      def lunar_phase(time): 
        sun = get_sun(time)
        moon = get_moon(time)
        elongation = sun.separation(moon) 
        lunar_angle = np.arctan2(sun.distance*np.sin(elongation),
                    moon.distance - sun.distance*np.cos(elongation))
        angle = lunar_angle/u.rad*180/np.pi # Necesitamos el ángulo en grados
        return angle

      # Masa de aire (en función del ángulo cenital)
      def X_airmass(zenith_angle):
        X = (1-0.972*np.sin(zenith_angle)**2)**(-0.5) 
        # Importante meter ángulo en radianes
        return X
      
      # BRILLO DEL CIELO SIN LA LUNA
      # Componente zodiacal: variación con el ángulo cenital
      zenith = self.zenith_angle_rad(time, location_observatory)
      mz = ms - 2.5*np.log10(X_airmass(zenith)) + extinction_coef*(X_airmass(zenith)-1)

      # Get moon data
      iers.conf.auto_download = False
      moon_data = get_moon(time)              # Datos de la Luna 
      lunar_angle = lunar_phase(time)         # Ángulo de fase de la luna en grados
      lunar_separation = self.lunar_distance_rad (time, moon_data)      # Distancia angular entre la Luna y el objeto
      a = moon_data.transform_to(AltAz(obstime = time, location = location_observatory))
      moon_zenith_angle_rad = np.pi/2 - a.alt.rad       # Ángulo cenital de la luna
      target_zenith_angle_rad = target.zenith_angle_rad(time, location_observatory)   # Ángulo cenital del objeto observado

      def B_moon_nanoLamberts(lunar_angle, lunar_separation, moon_zenith_angle, target_zenith_angle):
        illuminance = 10**(-0.4*(3.84 + 0.026*np.abs(lunar_angle) + 4e-9*lunar_angle**4)) # Illuminance of the moon inside the atmosphere
        f_rho = 10**(5.36)*(1.06 + (np.cos(lunar_separation))**2) + 10**(6.15-lunar_separation/40) # Función de scattering (rho es la distancia entre la luna y el objeto)
        B_moon = f_rho*illuminance*10**(-0.4*extinction_coef*X_airmass(moon_zenith_angle))*(1 - 10**(-0.4*extinction_coef*X_airmass(target_zenith_angle))) 
        return B_moon # Brillo de la luna en nano lamberts

      B_moon = B_moon_nanoLamberts(lunar_angle, lunar_separation, moon_zenith_angle_rad, target_zenith_angle_rad)
      # Pasamos a nano lamberts el valor obtenido para el cielo sin Luna en magnitudes
      # con la transformación de unidades dada por KRISCIUNAS
      B_sky = 34.08*np.exp(20.7233 - 0.92104 * mz) #Sky brighntess in nano Lamberts
      # Después, calculamos la variación en magnitud que produce el contraste con la luna
      # Al calcular la variación relativa podemos ignorar las unidades y no hacer el cambio inverso de los nano lamberts
      delta_m_moon = -2.5*np.log10((B_sky+B_moon)/B_sky)

      # Cuadro de warnings
      warnings_array = []
      # Luna llena
      if 0 < lunar_angle < 10:
        warnings_array.append("La Luna está prácticamente llena")
      # Objeto muy cerca de la luna
      if -0.175 < lunar_separation < 0.175:
        warnings_array.append("El objeto está muy cerca de la luna")
      # Masa de aire muy alta
      if X_airmass(target_zenith_angle_rad)>2:
        warnings_array.append("El valor de la masa de aire es muy alto")

      return mz, delta_m_moon, mz+delta_m_moon, X_airmass(target_zenith_angle_rad), warnings_array
      
      

In [0]:
# USO DE LA FUNCIÓN
#Fecha de observación en formato "YYYY-MM-DD" (El dia de la puesta de sol)
observation = Time("2020-01-23") + 1*u.d   #Se suma un día para tomar como referencia de nuestros cálculos las 0:00 de la noche de observación
print('Noche: {}'.format(observation))
#Posición del observatorio (latitud en grados, longitud en grados y altura en metros)
location_UAM  = EarthLocation(lat=40.54*u.deg, lon=-3.69*u.deg, height=720*u.m)             #Observatorio de la UAM (Madrid)
location_CAHA = EarthLocation(lat=37.22*u.deg, lon=-2.55*u.deg, height=2168*u.m)            #Observatorio +de Calar Alto (Almería)
location_ROMU = EarthLocation(lat=28.76*u.deg, lon=-17.88*u.deg, height=2326*u.m)           #Observatorio de Roque de los Muchachos (Islas Canarias)
location_MAKE = EarthLocation(lat=19.83*u.deg, lon=-155.47*u.deg, height=4215*u.m)          #Observatorio de Mauna Kea (Hawaii)
location_PARA = EarthLocation(lat=-24.63*u.deg, lon=-70.41*u.deg, height=2635*u.m)

#Seleccionamos un observatorio
location_observatory = location_UAM                                            
print('Posicion observatorio: {}'.format(location_observatory))

#Utc offsets
utc_offset_UAM = 1 #Madrid
utc_offset_CAHA = 1 #Almeria
utc_offset_ROMU = 0 #Canarias
utc_offset_MAKE = -10 #Hawaii
utc_offset_PARA = -3 #Chile

#Seleccionamos el horario local
utc_offset = utc_offset_UAM*u.h
print('UTC = ', utc_offset)

# Hora de observación
observation_hour = 3;                                    
observation_this_night = observation_hour*u.h - utc_offset + observation 
print(observation_this_night)

# Objeto a observar
target = []
name = "Objeto 1"
coordinates_object = SkyCoord('04 30 00 +15 00 00', unit=(u.hourangle, u.deg))
ra = coordinates_object.ra
dec = coordinates_object.dec
target=(Target(name, ra, dec))


# Parámetros del filtro (para el ejemplo cogemos los del U)
ms = 21.8
extinction_coef = 0.505
mu_sky, delta_mu_moon, mu_sky_final, masa_de_aire, warnings_array = target.brillo_cielo(location_observatory, observation_this_night, ms, extinction_coef)
print('Magnitud del cielo considerando la dependencia del ángulo cenital: '+ str(mu_sky))
print('Cambio de magnitud del cielo por la luna: ' +str(delta_mu_moon))
print('Magnitud del cielo considerando la dependencia del ángulo cenital y la luna: ' +str(mu_sky_final))
print('Masa de aire: ' + str(masa_de_aire))
print(warnings_array)







Noche: 2020-01-24 00:00:00.000
Posicion observatorio: (4844432.345036181, -312426.3477241187, 4124204.427031692) m
UTC =  1.0 h
2020-01-24 02:00:00.000


Magnitud del cielo considerando la dependencia del ángulo cenital: 21.63367646628114
Cambio de magnitud del cielo por la luna: -0.2018588587923284
Magnitud del cielo considerando la dependencia del ángulo cenital y la luna: 21.431817607488814
Masa de aire: 3.110176004502622
['El valor de la masa de aire es muy alto']
